# Time Series 를 DNN 용의 지도학습 dataset 으로 전환

## tf.data.Dataset

    - range(start, stop, step) - step 단계로 구분된 범위 값의 데이터 집합을 생성
    - window(size, shift) - input element를 윈도우 데이터셋으로 결합
    - flat_map(map_func) - 이 데이터세트 전체에 map_func를 매핑하고 그 결과를 flatten 
    - batch - 이 데이터 세트의 연속 elements를 batch로 결합
    - map - 이 데이터세트의 element 전체에 map_func를 매핑
    - shuffle - 이 데이터 세트의 elements를 무작위로 섞습니다.
    - prefetch(buffer_size) - buffer_size batch를 prefetch
    - from_tensor_slices - elements가 주어진 텐서의 조각인 Dataset을 생성.

## `range(start, stop, step)`
- step 단계로 구분된 범위 값의 데이터 집합을 생성

## window(size, shift, drop_remainder=True)

- ds.window : window dataset 생성

    - window: 그룹화 할 window size  
    - drop_remainder: 남은 부분을 버릴지 살릴지 여부 지정
    - shift는 1 iteration당 몇 element씩 이동할 것인지 지정
    
아래 코드는 TensorFlow를 사용하여 0부터 9까지의 정수로 구성된 데이터셋을 생성한 후, window 함수를 사용하여 크기가 5인 슬라이딩 윈도우로 변환하고 한 스텝씩 이동하도록 설정합니다. drop_remainder=True는 크기가 5보다 작은 나머지 윈도우를 삭제하도록 설정합니다.  

윈도우 데이터셋 ds의 각 원소를 출력하면 각 줄은 크기가 5인 슬라이딩 윈도우를 나타냅니다. 

### ds.take(count)

 ds 데이터셋에서 처음 3개의 슬라이딩 윈도우를 출력합니다. ds.take(3)를 사용하여 데이터셋에서 처음 3개의 윈도우를 가져옵니다. 그런 다음, for 루프를 사용하여 각 윈도우의 원소를 출력합니다. val.numpy()는 각 원소를 NumPy 배열 형태로 변환하여 출력합니다.

### filter(filter_fn)

아래 코드는 0부터 9까지의 정수로 구성된 데이터셋을 생성한 후, filter 함수를 사용하여 값이 3 초과인 원소만 포함하도록 필터링합니다. 그런 다음, window 함수를 사용하여 크기가 5인 슬라이딩 윈도우로 변환하고 한 스텝씩 이동하도록 설정합니다. 

### map

TensorFlow를 사용하여 1부터 10까지의 정수로 구성된 데이터셋을 생성한 후, map 함수를 사용하여 각 원소에 10을 곱합니다.

## flat_map(map_func)
- flat_map은 dataset에 함수를 apply해주고, 결과를 flatten하게 펼쳐 줍니다.

- 아래는 lambda 함수를 통해 5개의 batch를 읽어들인 뒤 flatten된 리턴값을 받습니다.

- ds.flat_map(lambda w: w.batch(5))은 아래의 coding 과 동일

## feature / label 분할

- Time Series Dataset을 만드려는 경우, train/label 값을 분류하는 용도로 `map()`을 활용할 수 있습니다.

- `x[:-1]`, `x[-1:]` 의 의도는 각 row의 마지막 index 전까지는 train data로, 마지막 index는 label로 활용하겠다는 의도입니다.

아래 코드는 flat_map 함수를 사용하여 각 윈도우의 요소들을 하나의 리스트로 결합하고 각 윈도우에 배치를 적용합니다.  
그런 다음 map 함수를 사용하여 각 윈도우를 입력 특성과 레이블로 분할합니다. 입력 특성은 윈도우의 처음 4개 요소이고, 레이블은 마지막 요소입니다.

## shuffle(buffer_size)

- 데이터세트는 buffer_size elements로 버퍼를 채운 다음 이 버퍼에서 elements를 무작위로 샘플링하여 선택한 elements를 새 elements로 바꿉니다.

- buffer_size: 새 데이터 세트가 샘플링할 이 데이터 세트의 element 수

- 완벽한 셔플링을 위해서는 데이터 세트의 전체 크기보다 크거나 같은 크기의 버퍼가 필요합니다. 예를 들어, 데이터 집합에 10,000 개의 element가 있지만 buffer_size가 1,000으로 설정된 경우 셔플은 처음에 버퍼의 처음 1,000 개 element 중 임의의 element 만 선택합니다.

- element가 선택되면 버퍼의 공간이 다음 element (즉, 1,001번째)로 대체되어 1,000 element 버퍼를 유지합니다.

- shuffle 의 위치는 input, label 분리 이전, 이후 모두 무방

### Sequence bias (시퀀스 편향)

- 시퀀스 편향은 시장 조사에서 목록의 특정 순서로 인해 사물, 사람 또는 그룹에 표시되는 편견 또는 호의로 정의됩니다. 예를 들어 응답자에게 가장 좋아하는 음식이 무엇인지 물어볼 경우, 피자가 항상 햄버거와 핫도그보다 먼저 순서대로 나열되어 있다면 응답자는 피자를 가장 먼저 읽었기 때문에 피자를 선택하는 경향이 더 높을 수 있습니다. 표시된 마지막 음식(핫도그)에 대해서도 마찬가지입니다. 이 경우 해당 음식이 목록의 마지막에 있기 때문에 선택될 가능성이 가장 낮을 수 있습니다.  

- 이러한 문제를 방지하기 위해 dataset을 shuffle 합니다.

### prefetch(buffer_size) 
- GPU memory 상으로 buffer_size batch를 prefetch

## repeat 

- 반복할 때마다 데이터셋을 다시 초기화
- element의 무한 시퀀스를 생성

repeat() 함수는 TensorFlow 데이터셋을 여러 번 반복하는 데 사용됩니다. 이 함수를 사용하면 데이터셋이 끝에 도달한 후 다시 처음부터 시작하여 데이터를 반복적으로 사용할 수 있습니다.

이 예제에서 repeat() 함수는 ds 데이터셋에 대해 적용되어 있습니다. 이 경우, repeat() 함수는 데이터셋의 원소들을 무한히 반복합니다. repeat()에 인수를 전달하면, 원하는 반복 횟수를 설정할 수 있습니다. 예를 들어, repeat(3)은 데이터셋의 원소들을 3번 반복하게 합니다.

###  `ds.repeat().batch()` 가 생성하는 것

repeat() 함수와 batch(3) 함수를 함께 사용하면, 먼저 데이터셋을 무한히 반복한 다음, 반복된 데이터셋의 원소들을 크기가 3인 배치로 묶는 것이 가능합니다.

이 예제에서 repeat() 함수는 데이터셋의 원소들을 무한히 반복하고, batch(3) 함수는 원소들을 크기가 3인 배치로 묶습니다. 반복된 데이터셋에서 배치를 만들기 때문에, 원소들이 반복되면서 여러 배치로 묶일 수 있습니다. 

## Pure Python Code 와 tf.data.Dataset 비교

### 순수 Python code 로 지도학습 data 생성

series_to_supervised() 함수는 주어진 시계열 데이터(series)를 입력 특성(X)과 레이블(y)로 변환하는데 사용됩니다. 이때 window_size는 과거의 관측값을 몇 개 사용할지를 결정하는 매개변수입니다.

함수는 다음과 같이 동작합니다:

1. 빈 리스트 X와 y를 생성합니다.  
2. 주어진 시계열 데이터를 순회하면서, 현재 인덱스(i)부터 i + window_size까지의 데이터를 seq_x로 저장합니다.  
3. seq_y에는 i + window_size 인덱스의 데이터를 저장합니다.  
4. 각 단계에서 seq_x를 X에 추가하고, seq_y를 y에 추가합니다.  
5. X와 y를 NumPy 배열로 변환하여 반환합니다.  

이 함수를 사용하면, 시계열 데이터를 슬라이딩 윈도우 방식으로 나누어 과거의 데이터를 입력 특성으로 사용하고, 다음 시점의 값을 예측하는 레이블로 사용할 수 있습니다.

## tf.data.Dataset 을 이용한 windowed dataset 생성

### Many-to-One output

windowed_dataset() 함수는 주어진 시계열 데이터(series)를 과거 관측값을 기반으로 미래 값을 예측하는 지도 학습 문제로 변환합니다. 함수에는 4개의 인자가 있습니다: series, window_size, batch_size, shuffle_buffer.

함수는 다음과 같은 작업을 수행합니다:

1. series를 입력으로 받아 tf.data.Dataset.from_tensor_slices() 함수를 사용하여 텐서플로 데이터셋을 생성합니다.  
2. 데이터셋에 window() 함수를 적용하여 window_size+1 크기의 슬라이딩 윈도우를 만들고, 윈도우가 1씩 이동하도록 설정합니다. drop_remainder=True를 설정하여 데이터셋의 끝 부분에서 원소가 부족한 윈도우를 삭제합니다.  
3. flat_map() 함수를 사용하여 각 윈도우를 하나의 배치로 결합합니다. 이때, 각 윈도우의 크기는 window_size+1 입니다. 4. map() 함수를 사용하여 각 배치를 입력 특성과 레이블로 분리합니다. 입력 특성은 각 배치의 처음부터 끝에서 두 번째 원소까지이고(w[:-1]), 레이블은 각 배치의 마지막 원소입니다(w[-1]). 이렇게 하면 many-to-one 출력 구조를 만듭니다.  
5. 전처리된 데이터셋을 반환합니다.

이 함수를 사용하면 주어진 시계열 데이터를 과거의 데이터를 입력 특성으로 사용하고, 다음 시점의 값을 예측하는 레이블로 사용할 수 있습니다.

### Many-to-Many output

windowed_dataset() 함수는 주어진 시계열 데이터(series)를 과거 관측값을 기반으로 미래 값을 예측하는 지도 학습 문제로 변환합니다. 이번에는 many-to-many 출력 구조로 변경되었습니다. 함수에는 4개의 인자가 있습니다: series, window_size, batch_size, shuffle_buffer.

함수는 다음과 같은 작업을 수행합니다:

1. series를 입력으로 받아 tf.data.Dataset.from_tensor_slices() 함수를 사용하여 텐서플로 데이터셋을 생성합니다.  
2. 데이터셋에 window() 함수를 적용하여 window_size+1 크기의 슬라이딩 윈도우를 만들고, 윈도우가 1씩 이동하도록 설정합니다. drop_remainder=True를 설정하여 데이터셋의 끝 부분에서 원소가 부족한 윈도우를 삭제합니다.  
3. flat_map() 함수를 사용하여 각 윈도우를 하나의 배치로 결합합니다. 이때, 각 윈도우의 크기는 window_size+1 입니다.  
3. map() 함수를 사용하여 각 배치를 입력 특성과 레이블로 분리합니다. 입력 특성은 각 배치의 처음부터 끝에서 두 번째 원소까지이고(w[:-1]), 레이블은 각 배치의 두 번째 원소부터 마지막 원소까지입니다(w[1:]). 이렇게 하면 many-to-many 출력 구조를 만듭니다.  
5. 전처리된 데이터셋을 반환합니다.

이 함수를 사용하면 주어진 시계열 데이터를 many-to-many 출력 구조로 변환할 수 있습니다. 